# Lab 8: Using functions as arguments to functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In mechanics (especially), we want to solve differential equations of the form:
$$ \frac{d \vec{\mathbf{x}}}{dt} = \vec{\mathbf{F}} (\vec{\mathbf{x}}) $$
In the previous lab we solved two differential equations:
$$ \frac{d}{dt} \left( \begin{array}{c} x \\ y \\ v_x \\ v_y \end{array} \right) = \left( \begin{array}{c} v_x \\ v_y \\ 0 \\ - 9.8 \end{array} \right) \qquad {\rm and} \qquad \frac{d}{dt} \left( \begin{array}{c} x \\ y \\ v_x \\ v_y \end{array} \right) = \left( \begin{array}{c} v_x \\ v_y \\ - b v_x \\ - 9.8 -  b v_y \end{array} \right) $$

With this in mind, we can create a function to represent the derivatives (the right-hand-side of the differential equations).  For example, the first one (projectile motion) can be written as a function:

In [ ]:
def F1(x_in):
    x, y, vx, vy = x_in
    
    derivative = np.zeros(4)
    derivative[:] = [vx, vy, 0, -9.8]
    return derivative

**Second Order Runge-Kutta Method** is a way of getting more bang for our computing buck.  Our original method just makes one calculation to take us from one time step to another.  What if we could do twice as many calculations, but get more than twice the accuracy from it?  That's the promise of Runge-Kutta methods.  A second order Runge-Kutta method (called Heun's method) is:

$$ k1_x = v_x \, \Delta t \qquad k1_y = v_y \, \Delta t$$
$$ k1_{vx} = a_x \, \Delta t \qquad k1_{vy} = a_y \, \Delta t$$

$$ k2_x = (v_x + k1_{vx}) \, \Delta t \qquad k2_y = (v_y + k1_{vy}) \, \Delta t$$
$$ k2_{vx} = a_x \, \Delta t \qquad k2_{vy} = a_y \, \Delta t$$

$$ x_f = x + 0.5 (k1_x + k2_x) \qquad y_f = y + 0.5 (k1_y + k2_y)$$
$$ v_{xf} = v_x + 0.5 (k1_{vx} + k2_{vx}) \qquad v_{yf} = v_y + 0.5 (k1_{vy} + k2_{vy})$$

Here, we're using $a_x = 0$ and $a_y = - 9.8$.  

This is a mess, we can actually simplify this with arrays:
$$\vec{\mathbf{k}}_1 = \vec{\mathbf{F}} (\vec{\mathbf{x}}) \, \Delta t$$
$$\vec{\mathbf{k}}_2 = \vec{\mathbf{F}} (\vec{\mathbf{x}} + \vec{\mathbf{k}}_1) \, \Delta t$$

$$\vec{\mathbf{x}}_f = \vec{\mathbf{x}} + \tfrac{1}{2} ( \vec{\mathbf{k}}_1 + \vec{\mathbf{k}}_2 )$$

In [ ]:
def time_step1(x_in, dt):
    k1 = F1(x_in) * dt
    k2 = F1(x_in + k1) * dt
    
    x_final = x_in + 0.5 * (k1 + k2)
    return x_final

Now we can solve the differential equations as we did before (with initial condition $x = 0,~y = 0,~v_x = 30,~v_y = 40$).

In [ ]:
time = np.linspace(0,8,100)
dt = time[1] - time[0]
x_step = np.zeros(len(time))
y_step = np.zeros(len(time))

x = 0
y = 0
vx = 30
vy = 40

x_array = np.zeros(4)
x_array[:] = [x, y, vx, vy]

x_step[0] = x_array[0]
y_step[0] = x_array[1]

for i in range(1,len(time)):
    x_array = time_step1(x_array, dt)
    x_step[i] = x_array[0]
    y_step[i] = x_array[1]
    
plt.figure()
plt.plot(x_step, y_step)
plt.show()

This is nice, but it seems doomed to error if we want to change our functions.  It would be useful if we could create a time step function that can change when we change our derivative function (to, for example, deal with different forces in this kinematics example).

**Using functions as arguments to functions**: Fortunately, we can treat functions as objects that can then act as the input to other functions. In the cell below, define a function, ``g(x)``, that takes an input and returns two times the input. Then, we know how to print the return value of a function, for example, ``print(g(2))``, but check out what happens when you try to ``print(g)``:

You'll notice that Python acknowledges that ``g`` is something... it's a function, and then it gives you an incomprehensible combination of numbers and letters. This is the location in memory that Python has stored the rules used to execute ``g(x)``. We can use this by creating a time stepping function that takes a function as an input to evaluate the derivatives:

In [ ]:
def time_step(x_in, dt, der):
    k1 = der(x_in) * dt
    k2 = der(x_in + k1) * dt
    
    x_final = x_in + 0.5 * (k1 + k2)
    return x_final

In the code, ``der`` represents a function, and ``der(x)`` is a function that takes an array as an input and returns an array of derivatives as the output. Use this ``time_step()`` function to solve the differential equation.  The way we use this function will involve: ``x_array = time_step(x_array, dt, F1)``.

Now, we solve the problem with air resistance and $b = 0.1$. 
- To do so, we need to create a new function to handle the derivatives.
- Once we create our new function, use the same ``time_step`` function to solve the differential equation and plot the result.

But if we want to choose a different value for $b$, with this method we need to completely re-run everything.  We should be able to use functions to alleviate this issue.

Create a new function:
- that now takes an array, ``x_in`` as an input as well as ``b`` as a second input
- and whose output is the array of derivatives

Once the new derivatives function is created, we need to create a new ``time_step`` function that has four inputs now:
- ``x_in``, the array of dependent variables (x, y, vx, vy)
- ``dt``, the $\Delta t$
- ``der``, the function that will take two inputs ``(x_in, b)``
- ``b``, the value of $b$ that will be input into the derivatives function

This allows us to solve the projectile motion with air resistance problem with minimal changing of the code.
- First, we want to make sure the code works. Fortunately, we know what the answer looks like when $b = 0$. Solve the projectile motion problem with $b = 0$ and confirm that your result is as expected by graphing y vs. x
- Then, solve the problem with $b = 0.1$

In order to compare the results, it's useful for us to create a function that solves the problem and graphs the result y vs. x. Your function should
- Take ``b`` as an input
- Solve the differential equations
- Plot y vs. x

Check your results for various values of $b$. Something looks amiss. It's hard to tell the difference for different $b$. The reason is that Python automatically re-sizes the y-axis to match your results. So, use ``plt.ylim( , )`` so that the zero air-resistance model is nicely plotted. Then, call your function with various values of $b$ to confirm that your code works.

When you are satisfied with your results, copy and paste all the code needed to create the two plots into the submission notebook and make sure it works!

**Exercise**: The Moon's orbit. We can consider the orbit of the Moon around the Earth with the differential equation
$$\displaystyle \frac{dx}{dt} = v_x \qquad \qquad \frac{d v_x}{dt} = a_x = - \frac{G M}{(x^2+y^2)^{3/2}} x$$
$$\displaystyle \frac{dy}{dt} = v_y \qquad \qquad \frac{d v_y}{dt} = a_y = - \frac{G M}{(x^2+y^2)^{3/2}} y$$
Here, $G M$ is the product of Newton's gravitational constant, $G$, and the mass of the Earth, $M$. To solve this problem, we need to find some initial conditions. Before that, we will make sure to choose to use SI units for position and time.
- Let's choose the initial conditions to be $(x, y) = (R, 0)$, where $R$ is the distance between the Moon and the Earth.
- We need to find an initial velocity, $(v_x, v_y) = (0, v)$, where $v$ is the speed of the Moon.

The last thing we need is a time scale. Find the amount of time needed for the moon to execute 1.5 orbits of Earth, let's divide this time up into 1000 steps, and solve the differential equation.

One thing we should do to evaluate the results is to look at energy conservation,
$$E = \frac{1}{2} m (v_x^2 + v_y^2) - \frac{G M m}{(x^2+y^2)^{1/2}},$$
here, $m$ is the mass of the moon. In total, your computational approach should
- Create a function for the derivatives
- Create a function that uses the derivatives to calculate the next time step using Heun's method.
- Solve the motion using 1000 steps, and create two separate plots: y vs. x and energy vs. time.
- We expect the Moon's orbit to be roughly circular, so make sure to use ``plt.figure(figsize=( , ))`` to create a square plotting region. However, we will also need to include ``plt.xlim( , )`` and ``plt.ylim( , )`` to make sure that both axes plot the same range.
- It's useful to have a marker for the location of the Earth. ``plt.plot(0, 0, 'o')`` will put a circle at the origin, the location of Earth.
- Increase the number of steps to a reasonable amount that runs in a reasonable amount of (computer) time

Now, let's see what happens if the Moon is moving slightly faster. Use the initial conditions $(x, y) = (R, 0)$ and $(v_x, v_y) = (0, 1.1 v)$. Use as many time steps as possible to solve the motion, and create plots of y vs. x and energy vs. time.

When you are satisfied with your results, copy and paste your results into the submission notebook and make sure it works!